## Module 3

This file is running on Databricks cluster: **DBR 9.1 LTS | Spark 3.1.2 | Scala 2.12**

Notebook has default language: **Python**

### Schema inference - semi-structured files

Make sure that the files for module3 are loaded and use Spark API file connection

In [0]:
import pyspark

In [0]:
spark.read.json("dbfs:/FileStore/module3/json1.json")

Out[15]: DataFrame[a: bigint, b: bigint, c: bigint, d: bigint, e: bigint]

In [0]:
spark.read.json("dbfs:/FileStore/module3/json1.json").printSchema()

root
-- a: long (nullable = true)
-- b: long (nullable = true)
-- c: long (nullable = true)
-- d: long (nullable = true)
-- e: long (nullable = true)

Actual file looks like and all types are long and nullable (!)
```
{"a":1, "b":2, "c":3}
{"e":2, "c":3, "b":5}
{"a":5, "d":7}
```

In [0]:
spark.read.json("dbfs:/FileStore/module3/json2.json")

Out[18]: DataFrame[a: string, b: bigint, c: double, d: bigint, e: bigint]

In [0]:
spark.read.json("dbfs:/FileStore/module3/json2.json").printSchema()

root
-- a: string (nullable = true)
-- b: long (nullable = true)
-- c: double (nullable = true)
-- d: long (nullable = true)
-- e: long (nullable = true)

Actual file looks like and all types are long and nullable (!)
```
{"a":1, "b":2, "c":3.1}
{"e":2, "c":3, "b":5}
{"a":"5", "d":7}
```

We can store results in dataFrame

In [0]:
df = spark.read.json("dbfs:/FileStore/module3/json2.json")
df.printSchema()
df.show()

root
-- a: string (nullable = true)
-- b: long (nullable = true)
-- c: double (nullable = true)
-- d: long (nullable = true)
-- e: long (nullable = true)

+----+----+----+----+----+
 a| b| c| d| e|
+----+----+----+----+----+
 1| 2| 3.1|null|null|
null| 5| 3.0|null| 2|
 5|null|null| 7|null|
+----+----+----+----+----+

In [0]:
# Read JSON file into dataframe
df = spark.read.format('org.apache.spark.sql.json') \
        .load("dbfs:/FileStore/module3/json2.json")

But we want to enforce schema to get correct import values; let's repeat for `json1.json` file

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,BooleanType,DoubleType

In [0]:
mojaShemica = StructType([
  StructField("a", IntegerType(), True),
  StructField("b", IntegerType(), True)])

In [0]:
spark.read.schema(mojaShemica).json("dbfs:/FileStore/module3/json1.json").show()

+----+----+
 a| b|
+----+----+
 1| 2|
null| 5|
 5|null|
+----+----+

In [0]:
spark.read.json("dbfs:/FileStore/module3/json1.json").printSchema()

root
-- a: long (nullable = true)
-- b: long (nullable = true)
-- c: long (nullable = true)
-- d: long (nullable = true)
-- e: long (nullable = true)

## Playing with data
Now let's create a more "interesting" JSON file

In [0]:
[{
  "RecordNumber": 2,
  "Zipcode": 1000,
  "ZipCodeType": "STANDARD",
  "City": "Ljubljana",
   "State":"SI"
},
{
  "RecordNumber": 10,
  "Zipcode": 3000,
  "ZipCodeType": "STANDARD",
  "City": "Celje",
   "State":"SI"
 },
{
  "RecordNumber": 32,
  "Zipcode": 100,
  "ZipCodeType": "STANDARD",
  "City": "Ljubljana",
   "State":"SI", 
   "Country":"Slovenia",
   "Lat":"46.0569",
   "Long":"14.5058"
 }]

Out[46]: [{'RecordNumber': 2,
 'Zipcode': 1000,
 'ZipCodeType': 'STANDARD',
 'City': 'Ljubljana',
 'State': 'SI'},
 {'RecordNumber': 10,
 'Zipcode': 3000,
 'ZipCodeType': 'STANDARD',
 'City': 'Celje',
 'State': 'SI'},
 {'RecordNumber': 32,
 'Zipcode': 100,
 'ZipCodeType': 'STANDARD',
 'City': 'Ljubljana',
 'State': 'SI',
 'Country': 'Slovenia',
 'Lat': '46.0569',
 'Long': '14.5058'}]

In [0]:
# Read multiline json file
multiline_df = spark.read.option("multiline","true") \
      .json("dbfs:/FileStore/module3/json3.json")
multiline_df.show() 

+---------+--------+-------+-------+------------+-----+-----------+-------+
 City| Country| Lat| Long|RecordNumber|State|ZipCodeType|Zipcode|
+---------+--------+-------+-------+------------+-----+-----------+-------+
Ljubljana| null| null| null| 2| SI| STANDARD| 1000|
 Celje| null| null| null| 10| SI| STANDARD| 3000|
Ljubljana|Slovenia|46.0569|14.5058| 32| SI| STANDARD| 100|
+---------+--------+-------+-------+------------+-----+-----------+-------+

Now let's read multiple files in this folder

In [0]:
# Read multiple files as listed names
df = spark.read.option("multiline","true").json(
    ['dbfs:/FileStore/module3/json3.json','dbfs:/FileStore/module3/json4_corrupt.json'])


This is why the files are corrupted. Missing comma and last comma must be removed.

```
[{
  "RecordNumber": 2,
  "Zipcode": 1000,
  "ZipCodeType": "STANDARD",
  "City": "Ljubljana",
   "State":"SI"
},
{
  "RecordNumber": 10,
  "Zipcode": 3000,
  "ZipCodeType": "STANDARD",
  "City": "Celje",
   "State":"SI"
 },
{
  "RecordNumber": 32,
  "Zipcode": 100,
  "ZipCodeType": "STANDARD",
  "City": "Ljubljana",
   "State":"SI", 
   "Country":"Slovenia",
   "Lat":"46.0569",
   "Long":"14.5058"
 }
{
  "RecordNumber": 104,
  "Zipcode": 89260,
  "ZipCodeType": "STANDARD",
  "City": "Seattle",
   "State":"WA",
   "Country":"USA"
 },
]
```

In [0]:
# Read multiple files as listed names and made json5 as corrected copy of json4
df = spark.read.option("multiline","true").json(['dbfs:/FileStore/module3/json3.json','dbfs:/FileStore/module3/json5.json'])


In [0]:
df.show()

+---------+--------+-------+-------+------------+-----+-----------+-------+
 City| Country| Lat| Long|RecordNumber|State|ZipCodeType|Zipcode|
+---------+--------+-------+-------+------------+-----+-----------+-------+
Ljubljana| null| null| null| 2| SI| STANDARD| 1000|
 Celje| null| null| null| 10| SI| STANDARD| 3000|
Ljubljana|Slovenia|46.0569|14.5058| 32| SI| STANDARD| 100|
 Seattle| USA| null| null| 104| WA| STANDARD| 89260|
Ljubljana| null| null| null| 2| SI| STANDARD| 1000|
 Celje| null| null| null| 10| SI| STANDARD| 3000|
Ljubljana|Slovenia|46.0569|14.5058| 32| SI| STANDARD| 100|
+---------+--------+-------+-------+------------+-----+-----------+-------+

Now let's try to read all files from a dedicated folder `dbfs:/FileStore/module3/input_files/`

In [0]:
# Read all JSON files from a folder
df = spark.read.option("multiline","true").json("dbfs:/FileStore/module3/input_files/*.json")
df.show()

+---------+--------+-------+-------+------------+-----+-----------+-------+
 City| Country| Lat| Long|RecordNumber|State|ZipCodeType|Zipcode|
+---------+--------+-------+-------+------------+-----+-----------+-------+
Ljubljana|Slovenia|46.0569|14.5058| 32| SI| STANDARD| 100|
 Seattle| USA| null| null| 104| WA| STANDARD| 89260|
 London| UK| null| null| 2| LN| STANDARD| 1000|
+---------+--------+-------+-------+------------+-----+-----------+-------+

Now let's infer schema. This schema will be user-specified and custom schema.

In [0]:
# Define custom schema
schema = StructType([
      StructField("RecordNumber",IntegerType(),True),
      StructField("Zipcode",IntegerType(),True),
      StructField("ZipCodeType",StringType(),True),
      StructField("City",StringType(),True),
      StructField("State",StringType(),True),
      StructField("LocationType",StringType(),True),
      StructField("Lat",DoubleType(),True),
      StructField("Long",DoubleType(),True),
      StructField("WorldRegion",StringType(),True),
      StructField("Country",StringType(),True),
      StructField("LocationText",StringType(),True)
  ])

In [0]:
df_with_schema = spark.read.schema(schema) \
        .json("dbfs:/FileStore/module3/json3.json")
df_with_schema.printSchema()
df_with_schema.show()

root
-- RecordNumber: integer (nullable = true)
-- Zipcode: integer (nullable = true)
-- ZipCodeType: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)
-- LocationType: string (nullable = true)
-- Lat: double (nullable = true)
-- Long: double (nullable = true)
-- Xaxis: integer (nullable = true)
-- Yaxis: double (nullable = true)
-- Zaxis: double (nullable = true)
-- WorldRegion: string (nullable = true)
-- Country: string (nullable = true)
-- LocationText: string (nullable = true)
-- Location: string (nullable = true)
-- Decommisioned: boolean (nullable = true)
-- TaxReturnsFiled: string (nullable = true)
-- EstimatedPopulation: integer (nullable = true)
-- TotalWages: integer (nullable = true)
-- Notes: string (nullable = true)

+------------+-------+-----------+----+-----+------------+----+----+-----+-----+-----+-----------+-------+------------+--------+-------------+---------------+-------------------+----------+-----+
RecordNumber|Zipcode|ZipCodeType|City|State|LocationType| Lat|Long|Xaxis|Yaxis|Zaxis|WorldRegion|Country|LocationText|Location|Decommisioned|TaxReturnsFiled|EstimatedPopulation|TotalWages|Notes|
+------------+-------+-----------+----+-----+------------+----+----+-----+-----+-----+-----------+-------+------------+--------+-------------+---------------+-------------------+----------+-----+
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
 null| null| null|null| null| null|null|null| null| null| null| null| null| null| null| null| null| null| null| null|
+------------+-------+-----------+----+-----+------------+----+----+-----+-----+-----+-----------+-------+------------+--------+-------------+---------------+-------------------+----------+-----+
only showing top 20 rows

Reading JSON files using Spark SQL

In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,BooleanType,DoubleType
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("module3") \
    .getOrCreate()

In [0]:
# Create a table from Parquet File
spark.sql("CREATE OR REPLACE TEMPORARY VIEW json3 USING json OPTIONS (path 'dbfs:/FileStore/module3/json3.json')")

Out[85]: DataFrame[]

In [0]:
df2 = spark.sql("select * from json3")

In [0]:
# PySpark write Parquet File
# referring to dataframe called df!
df.write.mode('Overwrite').json("dbfs:/FileStore/module3/output/res.json")

## Getting data from Source

Covered in Module 2

## Moving Data Around

### Using fs

In [0]:
# Default location for %fs is root
# Creating fol
%fs ls /tmp/
%fs mkdirs /tmp/my_cloud_dir
%fs cp /tmp/test_dbfs.txt /tmp/file_b.txt

In [0]:
# locate folder
# mount data
# read data
import os 
os.listdir('/dbfs/tmp')
dbutils.fs.ls("/mnt/mymount") 
df = spark.read.text("dbfs:/mymount/my_file.txt")

In [0]:
%scala

import scala.sys.process._

// Write a file using the local file API (over the FUSE mount).
dbutils.fs.put("file:/dbfs/tmp/test", "test-contents")

// Flush to persistent storage.
"sync /dbfs/tmp/test" !

// Read the file using "dbfs:/" instead of the FUSE mount.
dbutils.fs.head("dbfs:/tmp/test")

## Working with SQL

In [0]:
%scala

import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Basic Transformations").
    master("yarn").
    getOrCreate

General commands with scala

In [0]:
%sql
DROP DATABASE SQLBits2022 CASCADE


com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: org.apache.hadoop.hive.metastore.api.NoSuchObjectException: There is no database named sqlbits2022
	at org.apache.spark.sql.hive.HiveExternalCatalog.$anonfun$withClient$2(HiveExternalCatalog.scala:161)
	at org.apache.spark.sql.hive.HiveExternalCatalog.maybeSynchronized(HiveExternalCatalog.scala:112)
	at org.apache.spark.sql.hive.HiveExternalCatalog.$anonfun$withClient$1(HiveExternalCatalog.scala:150)
	at com.databricks.backend.daemon.driver.ProgressReporter$.withStatusCode(ProgressReporter.scala:377)
	at com.databricks.backend.daemon.driver.ProgressReporter$.withStatusCode(ProgressReporter.scala:363)
	at com.databricks.spark.util.SparkDatabricksProgressReporter$.withStatusCode(ProgressReporter.scala:34)
	at org.apache.spark.sql.hive.HiveExternalCatalog.withClient(HiveExternalCatalog.scala:149)
	at org.apache.spark.sql.hive.HiveExternalCatalog.dropDatabase(HiveExternalCatalog.scala:275)
	at org.apache.spark.sql.catalyst.catalog.ExternalCatalogWithListener.dropDatabase(ExternalCatalogWithListener.scala:61)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalogImpl.dropDatabase(SessionCatalog.scala:621)
	at com.databricks.sql.managedcatalog.ManagedCatalogSessionCatalog.dropDatabase(ManagedCatalogSessionCatalog.scala:306)
	at org.apache.spark.sql.execution.command.DropDatabaseCommand.run(ddl.scala:114)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:84)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:235)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3825)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:130)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:273)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:223)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:3823)
	at org.apache.spark.sql.Dataset. (Dataset.scala:235)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:689)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:684)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:694)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:526)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:266)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionConte

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS SQLBits2022

In [0]:
%sql
USE SQLBits2022

In [0]:
%sql
SHOW tables

database,tableName,isTemporary


In [0]:
%sql

DROP TABLE SQLUsers

In [0]:
%sql

CREATE TABLE SQLUsers (
    User_ID INT,
    User_Name STRING,
    Registration_date STRING,
    User_age INT,
    COVID_Status STRING,
    Country STRING
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

In [0]:
%sql
SHOW tables

database,tableName,isTemporary
sqlbits2022,sqlusers,false


In [0]:

%sql
-- fake Data :)

INSERT INTO SQLUsers VALUES
        (232, 'Stijn Wynants', '2021-12-20', 28, 'ok', 'BE'),
        (241, 'Terry and Simon', '2021-11-23', 33, 'ok', 'UK'),
        (22, 'Steve Jones', '2022-01-02', 43, 'ok', 'USA'), 
        (42, 'Cathrine Wilhelmsen', '2021-11-24', 33, 'ok', 'NO'),
        (52, 'Prathy Kamasani', '2021-12-03', 33, 'ok', 'UK');

In [0]:
%sql

SELECT * FROM SQLUsers

User_ID,User_Name,Registration_date,User_age,COVID_Status
232,Stijn Wynants,2021-12-20,28,ok
241,Terry and Simon,2021-11-23,33,ok
22,Steve Jones,2022-01-02,43,ok
42,Cathrine Wilhelmsen,2021-11-24,33,ok
52,Prathy Kamasani,2021-12-03,33,ok


In [0]:
%sql
-- overwrite complete table!
INSERT OVERWRITE SQLUsers VALUES
(232, 'Stijn Wynants', '2021-12-20', 31, 'ok', 'BE');
-- !!! MAke Sure to populate the table :) Thanks

In [0]:
%sql
SELECT * FROM SQLUsers

User_ID,User_Name,Registration_date,User_age,COVID_Status
232,Stijn Wynants,2021-12-20,31,ok


In [0]:
%sql 
-- importing directly into table

CREATE TABLE People_parquet (
  Age STRING,
  Name STRING
) USING parquet
OPTIONS (
  `parquet.bloom.filter.enabled#favorite_color` true,
  `parquet.bloom.filter.expected.ndv#favorite_color` 1000000,
  `parquet.enable.dictionary` true,
  `parquet.page.write-checksum.enabled` true
)

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table sqlbits2022.People_parquet already exists.
	at org.apache.spark.sql.execution.command.CreateDataSourceTableCommand.run(createDataSourceTables.scala:67)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:84)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:235)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3825)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:130)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:273)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:223)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:3823)
	at org.apache.spark.sql.Dataset. (Dataset.scala:235)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:689)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:684)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:694)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:526)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:266)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:261)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:258)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:50)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:305)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:297)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:50)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:503)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:689)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:681)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:522)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:634)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:427)
	at com.databricks.backend.daemon.driver.DriverWrapper.run

In [0]:
%sql
SELECT * FROM People_parquet

Age,Name


In [0]:
%sql

CREATE TEMPORARY VIEW parquetTable
USING org.apache.spark.sql.parquet
OPTIONS (
  path "dbfs:/people.parquet"
)

In [0]:
%sql
SELECT * FROM parquetTable

age,name
null,Michael
30,Andy
19,Justin


In [0]:
%sql
INSERT INTO People_parquet
SELECT * FROM parquetTable 

In [0]:
%sql
SELECT * FROM People_parquet
-- Check database for new tables! temp view: parquetTable should not be there! only: People_parquet

Age,Name
null,Michael
30,Andy
19,Justin


In [0]:
#for comparison
df = spark.sql("SELECT * FROM parquet.`dbfs:/people.parquet`")
df.show()

###  Spark SQL functions (count, countDistinct, Max, Min, Sum, SumDistinct, AVG)
###  Spark SQL statemetns, operations, WHERE

In [0]:
%sql

SELECT * FROM SQLUsers
-- WHERE Country IN ('UK', 'NO') LIMIT 2
-- WHERE Country NOT IN ('SI', 'BE')
-- WHERE Registration_date >= '2021-12-03'
-- WHERE YEAR(Registration_date) = 2022
-- WHERE USER_ID between 200 AND 250
WHERE User_name Like '%an%'

User_ID,User_Name,Registration_date,User_age,COVID_Status,Country
232,Stijn Wynants,2021-12-20,28,ok,BE
232,Stijn Wynants,2021-12-20,31,ok,BE
241,Terry and Simon,2021-11-23,33,ok,UK
52,Prathy Kamasani,2021-12-03,33,ok,UK


In [0]:
%sql

-- SELECT COUNT(*) as nof_all FROM SQLUsers
-- SELECT DISTINCT(User_ID) as nof_all_distinct FROM SQLUsers
-- SELECT countDistinct(User_ID) as nof_all_distinct FROM SQLUsers -- Nope
-- SELECT COUNT(DISTINCT(User_ID)) as nof_all_distinct FROM SQLUsers 
-- SELECT MAX(User_age), MIN(User_age), AVG(User_age) FROM SQLUsers
-- SELECT COUNT(DISTINCT(User_ID)) AS totalALL, Country FROM SQLUsers GROUP BY Country ORDER BY Country
-- SELECT SUM(CASE WHEN Country = 'UK' AND User_NAme like '%erry and Si%'THEN 2 ELSE 1 END) as NofPeople ,Country FROM SQLUSers GROUP BY Country

NofPeople,Country
2,BE
1,USA
3,UK
1,NO


In [0]:
%sql

-- using distinct not to get cartesian product
SELECT DISTINCT
   tab1.User_id
  ,tab1.User_Name 
  ,tab2.Country
 FROM SQLUsers AS tab1
INNER JOIN SQLUsers AS tab2
ON tab1.user_id = tab2.user_id
-- Check Data Profile, DAG, Query plan

User_id,User_Name,Country
22,Steve Jones,USA
42,Cathrine Wilhelmsen,NO
52,Prathy Kamasani,UK
232,Stijn Wynants,BE
241,Terry and Simon,UK


By using Spark functions, we get much more statistical and mathematical functions

Table partitioning using Hive with Parquet

In [0]:
%sql
-- Create table with partitions
CREATE TABLE SQLUsers_partitioned (
    User_ID INT,
    User_Name STRING,
    Registration_date STRING,
    User_age INT,
    COVID_Status STRING,
    Country STRING
) PARTITIONED BY (Month_registered STRING)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','



In [0]:
%sql
-- adding partitions
 
ALTER TABLE SQLUsers_partitioned ADD PARTITION (Month_registered='2022-04')

In [0]:
%sql
-- Adding new partitions for upcoming months

ALTER TABLE SQLUsers_partitioned ADD
    PARTITION (Month_registered='2022-05')
    PARTITION (Month_registered='2022-06')
    PARTITION (Month_registered='2022-07')

In [0]:
%sql
-- inserting into partitions

INSERT INTO TABLE SQLUsers_partitioned PARTITION (Month_registered='2022-05')
  SELECT * FROM SQLUsers WHERE Registration_date LIKE '%2022-05%'

Inserting from CLI 
// Pseudo code
```
rm -rf ~/registrations
mkdir -p ~/registrations

grep 2022-05 /Raw_zone/SQLBitsData/registrations/part-00000 > ~/registrations/Registrations_2022_05
grep 2022-06 /Raw_zone/SQLBitsData/registrations/part-00000 > ~/registrations/Registrations_2022_06
grep 2022-07 /Raw_zone/SQLBitsData/registrations/part-00000 > ~/registrations/Registrations_2022_07
```